## 文本摘要

假设有一整套文本文档语料库，其范围从句子到段落，你的任务是尝试从中获得有意义的见解。以下是可以对文本文档进行的一些操作：
* 提取文档中关键影响短语
* 提取文档中存在的各种不同的概念或主题
* 总结文件，以提供保留着整个语料库重要部分的要点。

我们将讨论三种主要的概念和技术：
* 关键短语提取
* 主题模型
* 自动文摘

### 几个重要概念

#### 文档

文档通常是一个包含完整文本数据的实体，包含可选的标题和其他元数据信息。语料库通常由一系列文档组成。这些文档可以是简单的句子或完整的文本信息段落。分词语料库指的是每个文档被分词化或分解成标识的语料库，其中标识通常是单词。

#### 文本规范化

我呢本规范化是通过技术来清洗、规范化和标准化文本数据的过程，譬如删除特殊符号和字符、去除多余的HTML标签、移除停用词、校正拼写(中文应该是修改错别字)、词干提取和词形还原

#### 特征提取

特征提取是我们从原始文本数据中提取有意义的特征或属性，以将其提供给统计或机器学习算法的过程。

#### 特征矩阵

特征矩阵通常是指从文档集合到特征的映射，其中每行表示文档，每列表示具体特征，特征通常是一个单词或一组单词。我们将通过特征提取后的特征矩阵来表达文档或句子的集合，并且将在后面的实例中经常在统计和机器学习技术之中应用这些矩阵。

#### 奇异值分解

奇异值分解（SVD）是线性代数的一种技术，他在摘要法中经常使用。SVD是实数或复数矩阵的因子分解过程。我们将使用源自scipy的一个很好的实现来提取顶部的k个奇异值，并返回相应的U、S和V矩阵。在utils.py文件中我们将使用以下代码段：

In [2]:
from scipy.sparse.linalg import svds
def low_rank_svd(matrix,singular_count = 2):
    u,s,vt = svds(matrix,k = singular_count)
    return u,s,vt

将在主题建模以及有关文本摘要的中使用该函数。

### 文本规范化

该过程的主要步骤包括
* 句子提取
* 取消HTML转移序列
* 扩展缩写词
* 文本还原
* 删除特殊字符
* 删除停用词
在第一步中我们将在收到的文本文档中，执行：删除其换行符，解析文本，将其转换为ASCII格式，并将其分解成其句子成分。

该函数代码如下所示：

In [4]:
def parse_document(document):
    document = re.sub('\n',' ',document)
    if isinstance(document,str):
        document = document
    elif isinstance(document,unicode):
        return unicodedata.normalize('NFKD',document).encode('ascii','ignore')
    else:
        raise ValueError('Document is not string or unicode!')
    document = document.strip()
    sentences = nltk.sent_tokenize(document)
    sentences = [sentence.strip() for sentence in sentences]
    return sentences

步骤二涉及被转义或编码的非转义特殊HTML字符。我们使用以下函数来取消他们的转义，并将它们恢复到原来未转义形式，这样便可在后续阶段规范化他们：


In [1]:
from HTMLParser import HTMLParser
html_parser = HTMLParser()
def unescape_html(parser,text):
    return parser.unescape(text)

我们要在最终规范化函数中用参数表示词形还原操作，以使其可选，因为在某些情况下我们不想使用词形还原。

In [4]:
def normalize_corpus(corpus,lemmatize = True ,tokenize = False):
    normalize_corpus = []
    for text in corpus:
        text = html_parser.unescape(text)
        text = expand_contractions(text,CONTRACTION_MAP)
        if lemmatize:
            text = lemmatize_text(text)
        else:
            text = text.lower()
        text = remove_special_characters(text)
        text = remove_stopwords(text)
        if tokenize:
            text = tokenize_text(text)
            normalize_corpus.append(text)
        else:
            normalize_corpus.append(text)
            
    return normalize_corpus

我们将使用该函数来满足大部分的规范化需要

### 特征提取

使用泛型函数从文本数据中执行各种类型的特征提取。将要使用的特征类型如下：
* 基于词项次数的二值特征
* 基于词袋模型的频率特征
* TF-IDF权重模型

我们将使用下面的函数实现从文本文档中提取特征

In [6]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
def build_feature_matrix(documents,feature_type = 'frequency'):
    feature_type = feature_type.lower().strip()
    if feature_type == 'binary':
        vectorizer = CountVectorizer(binary = True,min_df = 1,ngram_range = (1,1))
    elif feature_type == 'frequency':
        vectorizer = CountVectorizer(binary = False,min_df = 1,ngram_range = (1,1))
    elif feature_type == 'tfidf':
        vectorizer = TfidfVectorizer(min_df = 1,ngram_range = (1,1))
    else:
        raise Exception("Wrong feature type entered.Possible values:'binary','frequence','tfidf'")
    feature_matrix = vectorizer.fit_transform(documents).astype(float)
    return vectorizer,feature_matrix

## 关键短语提取

从非结构化文本文档中提取重要信息的最简单但最强大的技术之一是关键短语提取，关键短语提取也称为词项提取，定义为非结构化文本提取关键重要和相关词项或短语的过程或技术，使得文本文档的核心论题或主题涵盖在这些关键短语中。这种技术属于信息检索和提取的广泛领域。关键短语提取可以应用在许多领域，包括：
* 语义网
* 基于查询的搜索引擎和爬虫
* 推荐系统
* 标注系统
* 文档相似性
* 翻译


有各种各样的关键短语提取方法，下面介绍两种：
* 搭配
* 基于权重标签的短语提取

### 搭配

有各种方法提取搭配，最好方法之一是使用一种n元分词分组或分割方法，我们从语料库中构造n元分词分组，计算每个n元分词的频率，并根据他们的出现频率进行排序以得到最频繁的n元分词搭配。

我们使用古滕堡语料库的书的一部分内容作为我们的语料库

In [21]:
from nltk.corpus import gutenberg
from normalization3 import normalize_corpus
import nltk
from operator import itemgetter
#下载语料
alice = gutenberg.sents(fileids='carroll-alice.txt')
alice = [' '.join(ts) for ts in alice]
#在python3里filter不再返回list，需要强转
norm_alice =list(filter(None, normalize_corpus(alice, lemmatize=False)))
#打印第一行
print(norm_alice[0])

alice adventures wonderland lewis carroll 1865


In [22]:
def flatten_corpus(corpus):
    return ' '.join([document.strip() for document in corpus])

In [33]:
def compute_ngrams(sequence,n):
    return zip(*[sequence[index:] for index in range(n)])

In [34]:
print(compute_ngrams([1,2,3,4],2))

以下代码帮助我们获得最大的n元分词：

In [37]:
def get_top_ngrams(corpus,ngram_val = 1,limit = 5):
    corpus = flatten_corpus(corpus)
    tokens = nltk.word_tokenize(corpus)
    
    ngrams = compute_ngrams(tokens,ngram_val)
    ngrams_freq_dist = nltk.FreqDist(ngrams)
    sorted_ngrams_fd = sorted(ngrams_freq_dist.items(),key = itemgetter(1),reverse=True)
    sorted_ngrams = sorted_ngrams_fd[:limit]
    sorted_ngrams = [(' '.join(text),freq) for text,freq in sorted_ngrams]
    return sorted_ngrams

In [38]:
get_top_ngrams(corpus = norm_alice,ngram_val=2,limit = 10)

[('said alice', 123),
 ('mock turtle', 56),
 ('march hare', 31),
 ('said king', 29),
 ('thought alice', 26),
 ('white rabbit', 22),
 ('said hatter', 22),
 ('said mock', 20),
 ('said caterpillar', 18),
 ('said gryphon', 18)]

In [39]:
get_top_ngrams(corpus=norm_alice,ngram_val=3,limit=10)

[('said mock turtle', 20),
 ('said march hare', 10),
 ('poor little thing', 6),
 ('little golden key', 5),
 ('certainly said alice', 5),
 ('white kid gloves', 5),
 ('march hare said', 5),
 ('mock turtle said', 5),
 ('know said alice', 4),
 ('might well say', 4)]

In [40]:
from nltk.collocations import BigramCollocationFinder
from nltk.collocations import BigramAssocMeasures
finder = BigramCollocationFinder.from_documents([
    item.split()
    for item
    in norm_alice
])
bigram_measures = BigramAssocMeasures()

In [41]:
finder.nbest(bigram_measures.raw_freq,10)

[('said', 'alice'),
 ('mock', 'turtle'),
 ('march', 'hare'),
 ('said', 'king'),
 ('thought', 'alice'),
 ('said', 'hatter'),
 ('white', 'rabbit'),
 ('said', 'mock'),
 ('said', 'caterpillar'),
 ('said', 'gryphon')]

In [42]:
finder.nbest(bigram_measures.pmi,10)

[('abide', 'figures'),
 ('acceptance', 'elegant'),
 ('accounting', 'tastes'),
 ('accustomed', 'usurpation'),
 ('act', 'crawling'),
 ('adjourn', 'immediate'),
 ('adoption', 'energetic'),
 ('affair', 'trusts'),
 ('agony', 'terror'),
 ('alarmed', 'proposal')]

In [43]:
from nltk.collocations import TrigramCollocationFinder
from nltk.collocations import TrigramAssocMeasures
finder = TrigramCollocationFinder.from_documents([
    item.split()
    for item
    in norm_alice
])
trigram_measures = TrigramAssocMeasures()
finder.nbest(trigram_measures.raw_freq,10)

[('said', 'mock', 'turtle'),
 ('said', 'march', 'hare'),
 ('poor', 'little', 'thing'),
 ('little', 'golden', 'key'),
 ('march', 'hare', 'said'),
 ('mock', 'turtle', 'said'),
 ('white', 'kid', 'gloves'),
 ('beau', 'ootiful', 'soo'),
 ('certainly', 'said', 'alice'),
 ('might', 'well', 'say')]

In [44]:
finder.nbest(trigram_measures.pmi,10)

[('accustomed', 'usurpation', 'conquest'),
 ('adjourn', 'immediate', 'adoption'),
 ('adoption', 'energetic', 'remedies'),
 ('ancient', 'modern', 'seaography'),
 ('apple', 'roast', 'turkey'),
 ('arithmetic', 'ambition', 'distraction'),
 ('brother', 'latin', 'grammar'),
 ('canvas', 'bag', 'tied'),
 ('cherry', 'tart', 'custard'),
 ('circle', 'exact', 'shape')]

### 基于权重标签的短语提取